Використовуючи Python та бібліотеку pandas, порахуйте середній за весь період конверт користувачів з тріального періоду (ті, які заплатили перший платіж) в успішно сплачений другий платіж. Для розрахунку метрики включайте лише користувачів, які мають підписку *tenwords_1w_9.99_offer*.

In [1]:
import pandas as pd

In [2]:
transactions = pd.read_csv('input_files/file1_transactions.csv')
transactions['purchase_date'] = pd.to_datetime(transactions['purchase_date'])
transactions.head(10)

,user_id,product_id,refunded,purchase_date,country_code,media_source
0,2589061,tenwords_1w_7.99_7free,False,2020-09-27,JO,Facebook Ads
1,2604470,tenwords_1w_9.99_offer,False,2020-09-29,UK,Facebook Ads
2,2274467,tenwords_1w_9.99_offer,False,2020-08-18,Organic/Unknown,Organic/Unknown
3,2488755,tenwords_lifetime_limited_49.99,False,2020-09-14,PE,Facebook Ads
4,2571294,tenwords_1w_9.99_offer,False,2020-09-25,US,Google Ads
5,2217390,tenwords_1w_9.99_offer,False,2020-09-05,MX,Facebook Ads
6,2086438,tenwords_1w_9.99_offer,False,2020-07-28,US,Facebook Ads
7,2193321,tenwords_1w_9.99_offer,False,2020-09-01,CO,Facebook Ads
8,2330741,tenwords_1w_9.99_offer,False,2020-08-25,AE,Facebook Ads
9,2016953,tenwords_1w_7.99_7free,False,2020-07-11,AU,Facebook Ads


Для розрахунку конверсії  користувачів з тріального періоду (з першого платежу) в успішно сплачений другий платіж потрібно знайти кількість людей, що здійснили перший платіж, кількість людей, що здійснили перший і другий платежі та обчислити конверсію за формулою:

(кіль-ть людей з успішним другим платежем / кіль-ть людей з першим платежем) * 100%

*Так як в завданні сказано "всі користувачі, що присутні у датасеті виконали перше оформлення підписки за період від початку до кінця датасету", то кількість людей з першим платежем для цієї метрики буде рівна кількості всіх людей з підпискою *tenwords_1w_9.99_offer* незалежно від поля refunded. Для обрахунку кількості людей, що здійснили другий платіж потрібно знайти людей, які не здійснювали refund і для першої, і для другої оплати, тобто refunded == False.

Оберемо користувачів, які мають підписку *tenwords_1w_9.99_offer*, відсортуємо дані за id користувача та датою оплати і створимо нову колонку з нумерацією оплат для одного користувача за часом.

In [3]:
transactions_9_99 = transactions[transactions["product_id"] == "tenwords_1w_9.99_offer"]
transactions_9_99 = transactions_9_99.sort_values(by=['user_id', 'purchase_date'])

transactions_9_99['transaction_number'] = transactions_9_99.groupby('user_id').cumcount() + 1
transactions_9_99.head(10)

,user_id,product_id,refunded,purchase_date,country_code,media_source,transaction_number
6970,27637,tenwords_1w_9.99_offer,False,2020-07-05,PL,Facebook Ads,1
38650,31549,tenwords_1w_9.99_offer,False,2020-08-25,CO,Facebook Ads,1
61235,31549,tenwords_1w_9.99_offer,False,2020-08-25,CO,Facebook Ads,2
47316,31921,tenwords_1w_9.99_offer,False,2020-08-25,FR,Facebook Ads,1
31438,34244,tenwords_1w_9.99_offer,False,2020-09-07,NL,Facebook Ads,1
45838,34491,tenwords_1w_9.99_offer,False,2020-07-15,Organic/Unknown,Organic/Unknown,1
46464,34491,tenwords_1w_9.99_offer,False,2020-07-22,Organic/Unknown,Organic/Unknown,2
57328,34491,tenwords_1w_9.99_offer,False,2020-08-22,Organic/Unknown,Organic/Unknown,3
62809,34491,tenwords_1w_9.99_offer,False,2020-08-22,Organic/Unknown,Organic/Unknown,4
25192,35450,tenwords_1w_9.99_offer,False,2020-08-20,KR,Facebook Ads,1


Знайдемо користувачів з першою оплатою: це всі користувачі, що мають transaction_number == 1.

In [4]:
first_payments = transactions_9_99[transactions_9_99['transaction_number'] == 1]
num_first_payments = first_payments.shape[0]

print(f"Кількість користувачів з першою оплатою tenwords_1w_9.99_offer: {num_first_payments}")
first_payments.head()

Кількість користувачів з першою оплатою tenwords_1w_9.99_offer: 18014


,user_id,product_id,refunded,purchase_date,country_code,media_source,transaction_number
6970,27637,tenwords_1w_9.99_offer,False,2020-07-05,PL,Facebook Ads,1
38650,31549,tenwords_1w_9.99_offer,False,2020-08-25,CO,Facebook Ads,1
47316,31921,tenwords_1w_9.99_offer,False,2020-08-25,FR,Facebook Ads,1
31438,34244,tenwords_1w_9.99_offer,False,2020-09-07,NL,Facebook Ads,1
45838,34491,tenwords_1w_9.99_offer,False,2020-07-15,Organic/Unknown,Organic/Unknown,1


Знайдемо користувачів з успішною першою і другою оплатою (дві найстаріші транзакції, де refunded == False).

Так як у списку з 1 і 2 оплатами не обов'язково може бути друга оплата у одного користувача, потрібно додатково перевірити чи точно у нього є дві оплати.

In [5]:
first_and_second_payments = transactions_9_99[(transactions_9_99['transaction_number'] == 1) 
                                           | (transactions_9_99['transaction_number'] == 2)]
first_and_second_payments = first_and_second_payments[first_and_second_payments['refunded'] == False]

successful_two_payments = first_and_second_payments.groupby('user_id').filter(lambda x: len(x) == 2)
num_successful_two_payments = successful_two_payments['user_id'].nunique()

print(f"Кількість користувачів з успішною першою і другою оплатою tenwords_1w_9.99_offer: {num_successful_two_payments}")
successful_two_payments.head()

Кількість користувачів з успішною першою і другою оплатою tenwords_1w_9.99_offer: 7122


,user_id,product_id,refunded,purchase_date,country_code,media_source,transaction_number
38650,31549,tenwords_1w_9.99_offer,False,2020-08-25,CO,Facebook Ads,1
61235,31549,tenwords_1w_9.99_offer,False,2020-08-25,CO,Facebook Ads,2
45838,34491,tenwords_1w_9.99_offer,False,2020-07-15,Organic/Unknown,Organic/Unknown,1
46464,34491,tenwords_1w_9.99_offer,False,2020-07-22,Organic/Unknown,Organic/Unknown,2
25192,35450,tenwords_1w_9.99_offer,False,2020-08-20,KR,Facebook Ads,1


Обчислимо конверсію за формулою:

(кіль-ть людей з успішним другим платежем / кіль-ть людей з першим платежем) * 100%

In [6]:
conversion_trial_to_second_transaction = (num_successful_two_payments / num_first_payments) * 100

print(f"Кількість користувачів з першою оплатою: {num_first_payments}\n"
      f"Кількість користувачів з успішною першою і другою оплатою: {num_successful_two_payments}\n"
      f"Конверсія користувачів з тріального періоду в успішно сплачений другий платіж: {conversion_trial_to_second_transaction:.2f}%")

Кількість користувачів з першою оплатою: 18014
Кількість користувачів з успішною першою і другою оплатою: 7122
Конверсія користувачів з тріального періоду в успішно сплачений другий платіж: 39.54%


Бачимо, що конверсія склала 39.54%, що є досить високим показником.